<a href="https://colab.research.google.com/github/hellocml/Python/blob/main/jeju_cafe_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

제주 카페 분포 그리기

[제주카페데이터 다운로드](https://github.com/hellocml/Python/blob/main/cafe.csv)

출처: 공공데이터포털

## 1. 필요한 라이브러리 불러오기

### 라이브러리(Library)는 우리가 코딩할 때 자주 쓰는 기능들을 미리 모아 놓은 도구 상자
### 라이브러리는 누군가가 만들어둔 똑똑한 코드 꾸러미를 우리가 import해서 쉽게 쓸 수 있음
---



In [ ]:
import pandas as pd    # 표 형태의 데이터를 다루는 도구
import numpy as np     # numpy는 숫자계산을 도와주는 도구(계산기 역할)
import seaborn as sns  # seaborn은 통계 시각화 도구

%matplotlib inline     # 그래프를 코드 아래에 바로 출력되게 해주는 역할

## 2. 시각화를 위한 폰트 설정

### (한글이 보이게 도와주는 준비 작업)

### 보통 컴퓨터에서는 영어가 잘 나오지만, 그래프 그릴때 한글이 깨지거나 안나올수 있음
### 특히 Google Colab은 인터넷에서 작동하는 서비스라 처음에는 한글 폰트가 설치되어 있지 않음


In [ ]:
# 한글폰트 사용을 위해 설치
!pip install koreanize-matplotlib

import koreanize_matplotlib

In [ ]:
# 1. 한글 폰트 설치 (Colab 전용)
!apt-get update -qq
!apt-get install -y fonts-nanum

# 2. matplotlib 설정
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib as mpl
from IPython.display import set_matplotlib_formats

# 한글 폰트 적용
mpl.rc('font', family='NanumGothic')
mpl.rcParams['axes.unicode_minus'] = False  # 음수 깨짐 방지

# 디스플레이 설정 (Retina)
set_matplotlib_formats('retina')  # Retina 디스플레이용
%matplotlib inline


##3. 데이터 로드하기

### 제주 카페 정보를 엑셀처럼 표로 불러오기

In [ ]:
url = "https://raw.githubusercontent.com/hellocml/Python/main/cafe.csv"    #  파일이 저장된 인터넷 주소(URL)
df = pd.read_csv(url)   #url에 있는 파일을 읽어서 df라는 이름의 데이터표(데이터프레임)로 저장

In [ ]:
df.shape   # 행, 열 형

## 4. 데이터 미리보기

### 제주 카페 정보 파일을 불러온 후, 이 데이터안에 무엇이 있는지 보기

In [ ]:
# head 로 데이터를 미리보기. (앞5행 미리보기)
df.head()

## 5. 데이터 요약하기


In [ ]:
# info로 데이터의 요약을 보기
df.info()

## 6. 그룹화된 요약값 보기 - value_counts

### 어느 지역에 카페가 가장 많을가? (행정동별, 시군구별)

In [ ]:
# 행정동별 카페수 세기
df["행정동명"].value_counts()

In [ ]:
# 위에서 구한 결과를 판다스의 plot.bar()를 활용해 막대그래프로 그리기
df["행정동명"].value_counts().plot(kind='bar', figsize=(12, 6), width=0.5)

In [ ]:
# seaborn 의 countplot 을 사용해서 위에서 만든 df_cafe 데이터프레임의 시군구명을 시각화
plt.figure(figsize=(9, 6))
sns.countplot(data=df, y="시군구명")

In [ ]:
df["시군구명"].value_counts().plot.pie(figsize=(10, 10), autopct="%.1f%%", ylabel='')


## 7. 제주 카페 분포 보기

### 제주 카페 데이터를 지도에 표시
### 지도 위에 데이터(예: 위치, 경로, 밀도 등)를 시각적으로 표현

In [ ]:
!pip install folium  # folium은 지도 시각화 라이브러
import folium

In [ ]:
# Pandas 의 plot.scatter 를 통해 경도와 위도를 표시해 봅니다.
df[["경도", "위도", "시군구명"]].plot.scatter(
    x="경도", y="위도", figsize=(9, 7), grid=True)

In [ ]:
# seaborn의 scatterplot 을 통해 구별 경도와 위도를 표시하기(pandas와 달리 시군구명 별로 다른 컬러 표기 가능)
plt.figure(figsize=(16, 12))
sns.scatterplot(data=df,
                x='경도', y='위도', hue='시군구명')

In [ ]:
# geo_df 에 df_cafe를 copy() 로 복사
geo_df = df.copy()

In [ ]:
# 지도의 중심을 지정하기 위해 위도와 경도의 평균을 구하기
fmap = folium.Map(location=[geo_df['위도'].mean(), geo_df['경도'].mean()],
                 zoom_start=12)

for n in geo_df.index:
    # 팝업에 들어갈 텍스트를 지정
    # 상호명과 주소를 붙여서 출력
    popup_name = geo_df.loc[n, '상호명'] + ' - ' + geo_df.loc[n, '도로명주소']

    folium.Marker(
        location=[geo_df.loc[n,'위도'], geo_df.loc[n, '경도']],
        popup=popup_name,
        tooltip=popup_name
    ).add_to(fmap)


fmap